!!! Документ только для чтения, хотя можно и запустить !!!

# ЭТО БАЗА

In [ ]:
import napi
import napi_search

### ISearch

ISearch - это интерфейс высокого уровня. Через него организуется работа с основной частью кода посредством минимального количества затраченных усилий. Пока что он выполняет роль СТРОИТЕЛЯ (https://en.wikipedia.org/wiki/Builder_pattern), но когда-нибудь станет полноценным Singletone (https://en.wikipedia.org/wiki/Singleton_pattern)

In [ ]:
base = napi_search.ISearch()

### ISearch methods

В распоряжении ISearch есть 3 метода, которые возвращают:
1. Инструмент поиска коллекций
2. Инструмент поиска ассетов в коллекциях
3. Инструмент загрузки ассетов из облака(интернета)

In [ ]:
example = {
        'sat_names': ['landsat', 'hlsl', 'sentinel', 'hlss'],
        'catalog_filter': ['USGS_LTA', 'LPDAAC_ECS', 'LPCLOUD', 'ESA']
}

api = base.search_collections(**example) # -> имеет параметры sat_names и catalog_filter

api2 = base.search_items()

api3 = base.loader()

### ISearch -> search_collections()

Search-collections используется для поиска коллекций, в которых содержатся данные о спутнках, которые нам нужны. Возвращает Dataframe из pandas, т.к. это производительнее и быстрее.

In [ ]:
example = {
        'area': (42.171192, 45.04739, 42.18441, 45.053151),
        'date': '2024-09/2024-11'
}

dt = api.get_org_catalogs(**example)
# -> имеет параметры area и date
# area - область интереса
# date - временной промежуток: '2024-09/2024-11'.
# можно указать весь период, указав конкретную дату, которую можно раскрыть: '2024' - весь год

### ISearch -> search_items()

Используем для получения ассетов из баз данных Nasa. Вообще у них это называется гранула, они, как уже понятно, хранятся в коллекциях. Мы соответственно, используя pystac_client, совершаем запросы и получаем полные данные.

Важное примечание, это так просто, потому, что все данные, которые мы получаем, хранятся в формате JSON - благодаря STAC формату.

Соответственно, ассеты нам нужны, чтобы получать снимки в разных спектрах, так как своих у нас нет.

In [ ]:
example = {
    'collection': dt, 
    'max_items': 3
}

dt2_1 = api2.get_assets(**example) # Имеет множество параметров, тупо соответствующих оригинальным из библиотеки pystac_client
dt2_2 = api2.aget_assets() # Пока что не реализован, но это асинхронная версия get

Планируется реализовать асинхронность, так как при увеличении количества собираемых данных время, которое требуется равно O(2n). Так же потребляется и большое количество памяти. Привет, Python!

_P.S. А может просто кто-то не умеет им пользоваться..._

### ISearch -> loader

Класс, который реализует и поддерживает логику скачивания снимков из облака или из любой другой точки интернета. Главное двоичные content в ваших ха-ха response.

_P.S. Может использовать и ссылку, но эта магия уже из разряда храброго и капризного gdal_

In [ ]:
print(api3.download()) # -> параметры url, name

Но остановимся на самом сладком.

Для того, чтобы обратиться и скачать чего-то откуда-то оттуда нам нужно авторизоваться. Эту логику релизуется модуль napi

In [ ]:
config = napi.NasaAPIConfig('username', 'password')
client = napi.NasaAPIBase(config=config)

api3 = base.loader(base=client)

print(api3.download()) # -> Синхронная версия
#print(api3.adownload()) Ассинхронная версия (Пока не реализовано)

В приницпе, заботиться о том, что и как авторизуется на высоком уровне не требуется. Поэтому для деталей читайте следующий раздел.

_P.S. Да, можно было сделать и более высокий уровень (ха-ха куда еще выше...), чтобы создавать client в одну строчку, но пока что и так сойдет_

# Napi

Представляем вашему вниманию еще один builder pattern...

In [ ]:
api = napi.NasaAPIBase() # -> Нужен конфиг для полного функционирования config.

api.request() # -> NasaAPICall

api.session() # -> NasaSessionAPI

Как можно заметить, мы можем вызвать либо одиночный запрос, либо открыть сессию. Да, только это работает исключительно с Nasa и пока, только исключительно с lpdaac.

Работа этого модуля построена на том, что NasaAPIBase, являясь высоко уровневым интерфейсом, вызывает интерфейсы, который уже открывают двери к всем возможностям, что мы предусмотрели.

In [ ]:
api.request().get_oauth_profile()

api.request().get_oauth_token()

api.request().get_user_id()

In [ ]:
api.session()

api.session().create_session()

api.session().get_session()

api.session().close_session()

Reqeust - stateless, session - statefull

Так же, есть AuthManager, реализующий strategy (https://en.wikipedia.org/wiki/Strategy_pattern). Его работа более предсказуема, и особо не автономна.

Используется, для того, чтобы выбирать способ/метод авторизации. API Nasa имеет разные способы авторизации, как в принципе, и разные сайты для доступа к данным: dev и без dev.